In [24]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
    ExtraTreesRegressor,
)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import RationalQuadratic

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from datetime import datetime
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

#df = pd.read_csv('D:\PhD_ResearchWork\ASME_Journal\datasets\Final\LPBF_Dataset_polyExp(12features)Normalized.csv')

#output='depth'
#input_file='DataAug/LPBF_Final_Dataset/LPBF_Dataset_Normalized_Depth.csv'

output='width'
input_file='DataAug/LPBF_Final_Dataset/LPBF_Dataset_Normalized_Width.csv'

df = pd.read_csv(input_file)

X=df.iloc[:, :-1]
y=df.iloc[:, -1]





poly_model = make_pipeline(PolynomialFeatures(degree=2,include_bias=False), LinearRegression())




#('Polynomial Regression', poly_reg_model.fit(poly_features, y)),

ensemble_methods = [
    ('Gaussian Process', GaussianProcessRegressor(kernel=DotProduct() + WhiteKernel(), random_state=42)),
    ('Linear Regression', LinearRegression()),
    ('Polynomial Regression',poly_model),
    ('Support Vector Regression', SVR(kernel='linear', C=.5)),
    ('KNN', KNeighborsRegressor(n_neighbors=2)),
    ('Multi Layer Perception',
     MLPRegressor(activation='relu', hidden_layer_sizes=(10, 100), alpha=0.001, random_state=42, early_stopping=False)),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('AdaBoost', AdaBoostRegressor(n_estimators=100, random_state=42)),
    ('Bagging', BaggingRegressor(n_estimators=100, random_state=42)),
    ('Extra Trees', ExtraTreesRegressor(n_estimators=100, random_state=42)),
]

k = 10


results_list = []


for name, model in ensemble_methods:

    row = {'Model': name}

    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    
    # Use cross_val_score to get mean squared error and mean absolute error
    mse_scores = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    mae_scores = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')

    
    # Calculate metrics
    mse = mse_scores.mean()
    mae = mae_scores.mean()
    corr = r2_scores.mean()

    # Calculate additional metrics
    rmse = np.sqrt(mse)
    rae = mae / np.mean(np.abs(y - np.mean(y)))
    rrse = rmse / np.sqrt(np.mean((y - np.mean(y))**2))

    results_list.append({
        'Model': name,
        'R-squared': corr,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'RAE': rae,
        'RRSE': rrse
    })

   

results = pd.DataFrame(results_list)
# Save results to a CSV file

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = f'{output}_hp_model_comparison_results_{timestamp}.csv'

results.to_csv(filename, index=False)

#opt.to_csv('All_ML_models_Results_v3_Poly_Width(12features).csv',mode='w', index=False)

#opt.to_csv('All_ML_models_Results_depth.csv',mode='w', index=False)

